# Исследования по кластеризации текстовых данных

## Этап 1 - Подготовка 

In [1]:
import os
from pathlib import Path

DATA_DIR = Path("data")
if not DATA_DIR.exists():
    raise Exception("Data directory '%s' not found" % DATA_DIR.absolute())
    
OUT_DIR = Path("out_clasteri")
if not OUT_DIR.exists():
    OUT_DIR.mkdir()    

In [2]:
import numpy as np
import pandas as pd

## Этап 2 - Загрузка данных

In [3]:
complaints_file = os.path.join(DATA_DIR, "complaints.xlsx")

raw_data = pd.read_excel(complaints_file, sheet_name="complaints", index_col=None)

In [4]:
raw_data.head(3)

,ISSUENUM,TOPIC,DESCRIPTION,UNDER_TOPIC,REPORTER,ASSIGNEE,CREATOR,STATUS_ISSUE,CREATED,UPDATED,...,FIO_CLIENT,IIN,TEXT_CONTRACT_NUMBER,PHONE_NUMBER,INF_CLIENT_RESPONDING,INCIDENT_REGION,INCIDENT_RESOLVED,FIO_GUILT_WORK,AUTHOR_REGION,AUTHOR_SUBDIVISIONS
0,105578,ЖАЛОБА: на корреспонденцию банка,Клиент просит чтобы банк не отправлял смс о сп...,Cмс по предложениям Xsell,aoryntaeva,akalabaeva,aoryntaeva,Закрыта,2017-12-12 06:47:45,2017-12-12 12:02:50,...,АҚЖИГИТОВ БЕЙБІТ КЕНЖЕБАЙҰЛЫ,910429301833,0,7776121291,NaN,0,Не требует создания проблемы,NaN,Алматы,Call-center
1,105582,ЖАЛОБА: на корреспонденцию банка,ДД.\nПри оформлении кредита клиент предоставил...,Cмс по предложениям Xsell,atusipova,akalabaeva,atusipova,Закрыта,2017-12-12 08:34:06,2017-12-12 15:41:16,...,Тверитинова Анна Егоровна,761123400947,3708168706,87078452136 \t,NaN,0,Не требует создания проблемы,NaN,NaN,Call-center
2,105604,ЖАЛОБА: на обслуживание,Дарипова Айгуль Сабыргалиевна жалуется на мене...,NaN,azhalgaeva,zissayev,azhalgaeva,Закрыта,2017-12-12 09:27:28,2017-12-21 09:10:29,...,Дарипова Айгуль Сабыргалиевна,810420403003,0,7292250739,Не дозвон,"Мангистауская область,Бейнеу, ул. Ерконай, д. 1",Не требует создания проблемы,NaN,Актау,Call-center


In [5]:
print("[i] Размерность сырых данных: ", raw_data.shape)

[i] Размерность сырых данных:  (22840, 21)


## Этап 3 - Отбор признаков

In [6]:
valuable_columns = ["ISSUENUM", "TOPIC", "DESCRIPTION"]
raw_data = raw_data[valuable_columns] # Для исследования ограничимся ID 'ISSUENUM', меткой 'TOPIC' и содержанием 'DESCRIPTION'

print("[i] : Информация о данных")
raw_data.info() # Информация о данных

[i] : Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22840 entries, 0 to 22839
Data columns (total 3 columns):
ISSUENUM       22840 non-null int64
TOPIC          22805 non-null object
DESCRIPTION    22839 non-null object
dtypes: int64(1), object(2)
memory usage: 535.4+ KB


In [7]:
raw_data.head(3)

,ISSUENUM,TOPIC,DESCRIPTION
0,105578,ЖАЛОБА: на корреспонденцию банка,Клиент просит чтобы банк не отправлял смс о сп...
1,105582,ЖАЛОБА: на корреспонденцию банка,ДД.\nПри оформлении кредита клиент предоставил...
2,105604,ЖАЛОБА: на обслуживание,Дарипова Айгуль Сабыргалиевна жалуется на мене...


## Этап 4 - Проверка на дубли и удаление строк с отсутствующими данными

In [8]:
duplicated = raw_data.loc[raw_data.duplicated(subset=['ISSUENUM'], keep=False)]
print("[i] Количество дублей идентификаторов: ", duplicated['ISSUENUM'].count())

[i] Количество дублей идентификаторов:  0


In [9]:
raw_data = raw_data.dropna(subset=['TOPIC', 'DESCRIPTION']) # Удаляем строки с пустыми значениями в столбцах 'TOPIC' и 'DESCRIPTION'

print("[i] Размерность после удаления пустых", raw_data.shape)

[i] Размерность после удаления пустых (22804, 3)


## Этап 5 - Отбор "нужных" обращений

In [10]:
#Отберем только строки, содержащие жалобы, в которых столбец 'TOPIC' содержит различные сочетания слова "жалоба"
raw_data = raw_data[(raw_data.TOPIC.str.contains("ЖАЛОБА: ") |
                     raw_data.TOPIC.str.contains("Жалоба на ") |
                     raw_data.TOPIC.str.contains("Жалобы на "))] #, na=False

print("[i] Размерность данных с Жалобами", raw_data.shape)

[i] Размерность данных с Жалобами (19415, 3)


## Этап 6 - Очистка меток от "мусора"

In [11]:
import re
def label_cleaner(text):
        
    text = re.sub( r'ЖАЛОБА: на ', '', text) # Убрать из темы словосочетания 'ЖАЛОБА: на ',
    text = re.sub( r'ЖАЛОБА: ', '', text) # 'ЖАЛОБА: ',
    text = re.sub( r'Жалоба на ', '', text) # 'Жалоба на ',
    text = re.sub( r'Жалобы на ', '', text) # 'Жалобы на '
    
    text = str.strip(text.lower()) #Переводим в нижний регистр и убираем пробелы в начале и конце
    return  text

In [12]:
raw_data["TOPIC"] = [label_cleaner(t) for t in raw_data["TOPIC"]] # Почистим метки от "мусора"

print("[i] Количество категорий (меток):", len(set(raw_data["TOPIC"])))
print("[i] Множество категорий (меток):")
print(set(raw_data["TOPIC"]))

[i] Количество категорий (меток): 9
[i] Множество категорий (меток):
{'услугу "хранитель"', 'услугу "защита семьи"', 'обслуживание', 'карточные продукты', 'не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями', 'го/филиал/отделения/микроофисы/тт', 'обслуживание в терминалах', 'корреспонденцию банка', 'услугу "страховку"'}


## Этап 7 - Препроцессинг

In [13]:
#Пишем свой препроцессор (preprocessor)
from nltk.stem.snowball import SnowballStemmer
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    text = re.sub( r'https?://[\S]+', ' url ', text) # замена интернет ссылок
    text = re.sub( r'[\w\./]+\.[a-z]+', ' url ', text) 
 
    # text = re.sub( r'\d+[-/\.]\d+[-/\.]\d+', ' date ', text) # замена даты и времени
    # text = re.sub( r'\d+ ?гг?', ' date ', text) 
    # text = re.sub( r'\d+:\d+(:\d+)?', ' time ', text) 

    # text = re.sub( r'@\w+', ' tname ', text ) # замена имён twiter
    # text = re.sub( r'#\w+', ' htag ', text ) # замена хештегов

    text = re.sub( r'<[^>]*>', ' ', text) # удаление html тагов
    text = re.sub( r'[\W]+', ' ', text ) # удаление не словарных символов

    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    # stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
    #        'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    # remove = r'\b('+'|'.join(stw)+')\b'
    # text = re.sub(remove,' ', text)
    
    # text = re.sub( r'\b\w\b', ' ', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

#raw_data["DESCRIPTION"] = [text_cleaner(t) for t in raw_data["DESCRIPTION"]]
#raw_data["DESCRIPTION"].head(5)

## Этап 8 - Разделение на тренировочную и тестовую выборки

## Этап 9 - Обучение классификатора 

## Этап 10 - Тестирование

## Этап 11 - Сохранение результатов